# Alma Zohar binary classification model using Transfer Learning 

Designed to detect if Alma or Zohar using Transfer learning of the VGG16, ResNet50 and InceptionV3 models







---







##Arrange our data

###Import training set from My Drive

In [0]:
import os 
from google.colab import drive
drive.mount('/content/drive/')

###Extract data set

In [0]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/ML/AlmaZohar.zip", 'r')
zip_ref.extractall("/tmp/AlmaZohar")
zip_ref.close()

print(len(os.listdir('/tmp/AlmaZohar/Alma/')))
print(len(os.listdir('/tmp/AlmaZohar/Zohar/')))


###Equalize category sizes

In [0]:
import random

def even_data(cat1, cat2 ):
  len1 = len(os.listdir(cat1))
  len2 = len(os.listdir(cat2))
  if  len1 > len2:
    larger = cat1
    smaller = cat2
  else:
    larger = cat2
    smaller = cat1
  
  larger_len = len(os.listdir(larger))
  smaller_len = len(os.listdir(smaller))
  larger_image_list = os.listdir(larger)
  larger_image_list = random.sample(larger_image_list, len(larger_image_list)) # Shuffle list
  for idx, image in enumerate(larger_image_list):
    image_path = larger + image
    if os.path.getsize(image_path):
      if idx < (larger_len - smaller_len):
        os.remove(image_path)
      else:
        break
    else:
      print(image, " is zero length, so ignoring")



even_data  ('/tmp/AlmaZohar/Alma/', '/tmp/AlmaZohar/Zohar/') 
print(len(os.listdir('/tmp/AlmaZohar/Alma/')))
print(len(os.listdir('/tmp/AlmaZohar/Zohar/')))


### Arrange data for generators

In [0]:
from pathlib import Path


ALMA_SOURCE_DIR = "/tmp/AlmaZohar/Alma/"
TRAINING_ALMA_DIR = "/tmp/alma-vs-zohar/training/Alma/"
TESTING_ALMA_DIR = "/tmp/alma-vs-zohar/testing/Alma/"
ZOHAR_SOURCE_DIR = "/tmp/AlmaZohar/Zohar/"
TRAINING_ZOHAR_DIR = "/tmp/alma-vs-zohar/training/Zohar/"
TESTING_ZOHAR_DIR = "/tmp/alma-vs-zohar/testing/Zohar/"


Path(TRAINING_ALMA_DIR).mkdir(parents=True, exist_ok=True)
Path(TESTING_ALMA_DIR).mkdir(parents=True, exist_ok=True)
Path(TRAINING_ZOHAR_DIR).mkdir(parents=True, exist_ok=True)
Path(TESTING_ZOHAR_DIR).mkdir(parents=True, exist_ok=True)


In [0]:
import random
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  
  source_image_list = os.listdir(SOURCE)
  source_image_list = random.sample(source_image_list, len(source_image_list)) # Shuffle list
  num_source_images = len(os.listdir(SOURCE))
  split_num = round(num_source_images * SPLIT_SIZE)
  for idx, image in enumerate(source_image_list):
    image_path = SOURCE + image
    if os.path.getsize(image_path):
      if idx < split_num:
        copyfile(image_path, TRAINING + image)
      else:
        copyfile(image_path, TESTING + image)
    else:
      print(image, " is zero length, so ignoring")

#delete old files, if any
!find "/tmp/alma-vs-zohar/" -type f -delete

split_size = .7
split_data(ALMA_SOURCE_DIR, TRAINING_ALMA_DIR, TESTING_ALMA_DIR, split_size)
split_data(ZOHAR_SOURCE_DIR, TRAINING_ZOHAR_DIR, TESTING_ZOHAR_DIR, split_size)

Check the numbers

In [0]:
print('total training Alma images:', len(os.listdir(TRAINING_ALMA_DIR)))
print('total training Zohar images:', len(os.listdir(TRAINING_ZOHAR_DIR)))
print('total validation Alma images:', len(os.listdir(TESTING_ALMA_DIR)))
print('total validation Zohar images:', len(os.listdir(TESTING_ZOHAR_DIR)))


###Create training and validation generators

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input


HEIGHT = 150
WIDTH = 150

TRAINING_DIR = '/tmp/alma-vs-zohar/training/'
VALIDATION_DIR ='/tmp/alma-vs-zohar/testing'

train_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=preprocess_input,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')



validation_datagen = ImageDataGenerator(rescale=1/255)

        
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,  # This is the source directory for training images
        target_size=(HEIGHT, WIDTH),  # All images will be resized to 150x150
        batch_size=20,
        class_mode='binary')


# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,  # This is the source directory for training images
        target_size=(HEIGHT, WIDTH),  # All images will be resized to 150x150
        batch_size=20,
        class_mode='binary')

print( train_generator.class_indices)
print( validation_generator.class_indices)

## Build some models

###Simple CNN

####Build and compile

In [0]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(HEIGHT, WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
model.summary()

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])


    

####Train the model

In [0]:
NUM_EPOCHS = 50

history = model.fit_generator(train_generator,
                                      validation_data=validation_generator,
                                       epochs=NUM_EPOCHS,
                                      verbose=2)
    

###VGG16

####Fine tune on top of VGG16

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model


base_model = VGG16(weights=None, 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))


for layer in base_model.layers: 
  layer.trainable = False
base_model.summary()


for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

    
x = base_model.output
x = layers.Flatten()(base_model.output) #last_output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense (1, activation='sigmoid')(x)           

model = Model(inputs=base_model.input, outputs=x)


model.summary()




####Train the model - VGG16

In [0]:
from tensorflow.keras.optimizers import RMSprop

NUM_EPOCHS = 50


model.compile( optimizer=RMSprop(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])


history = model.fit_generator(train_generator,
                                      validation_data=validation_generator,
                                       epochs=NUM_EPOCHS,
                                      verbose=2) 

####fine tune

In [0]:
#Train last ResNet block
for layer in model.layers[:6]:
   layer.trainable = False
for layer in model.layers[6:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['acc'])

NUM_EPOCHS = 50

history = model.fit_generator(train_generator,
                                      validation_data=validation_generator,
                                       epochs=NUM_EPOCHS,
                                      verbose=2) 

###ResNet50

####Fine tune on top of  Resnet

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers


base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))


# for layer in base_model.layers: 
#   layer.trainable = False
# base_model.summary()

x = base_model.output
x = layers.Flatten()(base_model.output) #last_output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  

x = layers.Dense (1, activation='sigmoid')(x)           

model = Model(inputs=base_model.input, outputs=x)

#### Train our combiend model

In [0]:
from tensorflow.keras.optimizers import RMSprop

NUM_EPOCHS = 50

model.compile( optimizer=RMSprop(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])


history = model.fit_generator(train_generator,
                                      validation_data=validation_generator,
                                       epochs=NUM_EPOCHS,
                                      verbose=2)

####Fine tune if needed

In [0]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)
    

    
#Train last ResNet block
# for layer in model.layers[:7]:
#    layer.trainable = False
# for layer in model.layers[7:]:
#    layer.trainable = True
    
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['acc'])

NUM_EPOCHS = 50

history = model.fit_generator(train_generator,
                                      validation_data=validation_generator,
                                       epochs=NUM_EPOCHS,
                                      verbose=2) 

###Inception

####Fine tune on top of Inception

In [0]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K


# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

K.clear_session() #In order to keep layer names the same in every iteration


pre_trained_model = InceptionV3(include_top=False,
                               input_shape = (HEIGHT,WIDTH,3))

pre_trained_model.load_weights(local_weights_file)

# Make some layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers[:132]: #https://github.com/keras-team/keras/issues/9460#issuecomment-370736493
  layer.trainable = False
  
# Print the model summary
pre_trained_model.summary()


In [0]:


last_layer = pre_trained_model.get_layer("mixed5")
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output) # pre_trained_model.output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.4)(x)                  
x = layers.Dense (1, activation='sigmoid')(x)           

model = Model(inputs=pre_trained_model.input, outputs=x)

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

model.summary()





####Train the combined model

In [0]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True
      
      
      
callbacks = myCallback()
history = model.fit_generator(train_generator,
                              epochs=50,
                              verbose=2,
                              validation_data=validation_generator,
                              callbacks = [callbacks])

####Fine tune if needed

In [0]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(pre_trained_model.layers):
   print(i, layer.name)


###Plot the results

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

###Prediction

Let's now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, it will then upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

In [0]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150,3))
  x = image.img_to_array(img)
  x = x/255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=100)
  print(classes)
  if classes[0]>0.5:
    print(fn + " is a Zohar")
  else:
    print(fn + " is a Alma")
 